In [9]:
import pandas as pd
import urllib.parse as parse
from urllib.request import Request, urlopen
from urllib.error import HTTPError
import json

In [2]:
#naver map api key
client_id = '861z66ikys';    # 본인이 할당받은 ID 입력
client_pw = 'k5M5KmlwOFtZBg027Z5q6myw7f06R1p33I1gjviv';    # 본인이 할당받은 Secret 입력


In [3]:
pd.read_csv("전체차량전처리완료.csv")

,Unnamed: 0,차량번호,보고시각,속도,주소(지번),주소(POI),주행거리(m),월,일,시,분
0,1,81수0990,2024-02-01 00:03:53,0,서울특별시 중구 황학동 2504,서울특별시 중구 황학동 소라미용실(11m ↖),762,2,1,0,3
1,3,81수0990,2024-02-01 00:09:53,0,서울특별시 중구 황학동 641,서울특별시 중구 황학동 하늘드리움(9m ↖),983,2,1,0,9
2,4,81수0990,2024-02-01 00:12:53,0,서울특별시 중구 황학동 1014,서울특별시 중구 황학동 필승문방구(2m ↗),1034,2,1,0,12
3,5,81수0990,2024-02-01 00:15:53,0,서울특별시 중구 황학동 510,서울특별시 중구 황학동 태성가구(2m →),1082,2,1,0,15
4,7,81수0990,2024-02-01 00:21:53,0,서울특별시 중구 황학동 150,서울특별시 중구 황학동 수림주방(10m ↘),1324,2,1,0,21
...,...,...,...,...,...,...,...,...,...,...,...
28437,88880,800조2173,2024-03-31 23:47:39,0,서울특별시 중구 신당동 304-211,서울특별시 중구 신당동 삼보사(8m ↘),67319,3,31,23,47
28438,88881,800조2173,2024-03-31 23:50:40,4,서울특별시 중구 신당동 304-199,서울특별시 중구 신당동 CU(13m →),67395,3,31,23,50
28439,88882,800조2173,2024-03-31 23:53:39,7,서울특별시 중구 신당동 304-154,서울특별시 중구 신당동 IMC(2m ↑),67676,3,31,23,53
28440,88883,800조2173,2024-03-31 23:56:39,0,서울특별시 중구 신당동 228-17,서울특별시 중구 신당동 광희문성지(4m ↘),67789,3,31,23,56


In [10]:

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='


# 주소 목록 파일 (.csv)
data = pd.read_csv("전체차량전처리완료.csv")
# 주소 목록 파일 (.xlsx)
# data = pd.read_excel('파일명.xlsx')

# 네이버 지도 API 이용해서 위경도 찾기
geo_coordi = []     
for add in data['주소(지번)']:
    add_urlenc = parse.quote(add)  
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)   # json
            if response_body['addresses'] == [] :
                print("'result' not exist!")
                latitude = None
                longitude = None
            else:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
        else:
            print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

    geo_coordi.append([latitude, longitude])



'result' not exist!


KeyboardInterrupt: 

In [12]:
len(geo_coordi)

2332

In [ ]:
np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({"도로명": data['주소(지번)'].values,
                              "위도": np_geo_coordi[:, 0],
                              "경도": np_geo_coordi[:, 1]})

df_0990_xy=pd_geo_coordi
df_0990_xy.to_csv("0990_새벽")

In [18]:
import requests

def get_road_address(lat, lon, client_id=client_id, client_pw=client_pw):
    url = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_pw,
    }
    params = {
        "coords": f"{lon},{lat}",
        "sourcecrs": "epsg:4326",
        "orders": "roadaddr",
        "output": "json"
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status']['code'] == 0:
            result = data['results'][0]['region']
            address = f"{result['area1']['name']} {result['area2']['name']} {result['area3']['name']}"
            return address
        else:
            return "No results found"
    else:
        return f"Error: {response.status_code}"

# Example usage
latitude = 37.5665
longitude = 126.9780
address = get_road_address(latitude, longitude, client_id, client_pw)
print(address)



서울특별시 중구 태평로1가


In [24]:
df=pd.read_csv('../data/final_num_171.csv')
df

,Unnamed: 0,x,y
0,0,127.013725,37.555783
1,1,127.015149,37.560326
2,2,127.016254,37.559378
3,3,127.018157,37.561107
4,4,127.010355,37.563355
...,...,...,...
167,167,127.015517,37.558621
168,168,127.016363,37.558736
169,169,127.010366,37.562235
170,170,127.010508,37.564647


In [16]:
df['address'] = df.apply(lambda row: get_road_address(row['y'], row['x'], client_id, client_pw), axis=1)
df

,Unnamed: 0,x,y,address
0,0,127.013725,37.555783,No results found
1,1,127.015149,37.560326,No results found
2,2,127.016254,37.559378,No results found
3,3,127.018157,37.561107,No results found
4,4,127.010355,37.563355,No results found
...,...,...,...,...
167,167,127.015517,37.558621,서울특별시 중구 신당동
168,168,127.016363,37.558736,서울특별시 중구 신당동
169,169,127.010366,37.562235,No results found
170,170,127.010508,37.564647,No results found


In [22]:
get_road_address(37.558736,127.016363)

'서울특별시 중구 신당동'

In [25]:
import pandas as pd
import requests

# 예제 데이터프레임 생성

# 네이버 API 클라이언트 ID와 비밀번호


def get_road_address(lat, lon, client_id=client_id, client_pw=client_pw):
    url = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_pw,
    }
    params = {
        "coords": f"{lon},{lat}",
        "sourcecrs": "epsg:4326",
        "orders": "roadaddr",
        "output": "json"
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status']['code'] == 0:
            result = data['results'][0]['land']
            address = f"{result['name']} {result['number1']}-{result['number2']}" if result['number2'] else f"{result['name']} {result['number1']}"
            return address
        else:
            return "No results found"
    else:
        return f"Error: {response.status_code}"

# 데이터프레임에 도로명 주소 추가
df['address'] = df.apply(lambda row: get_road_address(row['y'], row['x'], client_id, client_pw), axis=1)

# 결과 출력
display(df)


,Unnamed: 0,x,y,address
0,0,127.013725,37.555783,No results found
1,1,127.015149,37.560326,No results found
2,2,127.016254,37.559378,No results found
3,3,127.018157,37.561107,No results found
4,4,127.010355,37.563355,No results found
...,...,...,...,...
167,167,127.015517,37.558621,청구로3길 15-2
168,168,127.016363,37.558736,다산로28길 57
169,169,127.010366,37.562235,No results found
170,170,127.010508,37.564647,No results found


In [26]:
import requests

# 네이버 API 클라이언트 ID와 비밀번호

def get_road_address(lat, lon, client_id, client_pw=client_pw):
    url = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_pw,
    }
    params = {
        "coords": f"{lon},{lat}",
        "sourcecrs": "epsg:4326",
        "orders": "roadaddr",
        "output": "json"
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status']['code'] == 0:
            result = data['results'][0]['land']
            address = f"{result['name']} {result['number1']}-{result['number2']}" if result['number2'] else f"{result['name']} {result['number1']}"
            return address
        else:
            print("API Response Data:", data)  # 디버깅을 위해 응답 데이터 출력
            return "No results found"
    else:
        print(f"HTTP Error: {response.status_code}")  # 디버깅을 위해 HTTP 오류 출력
        return f"Error: {response.status_code}"

# 테스트할 좌표
latitude = 37.555783
longitude = 127.013725

# 도로명 주소 가져오기
address = get_road_address(latitude, longitude, client_id, client_pw)
print("Address:", address)


API Response Data: {'status': {'code': 3, 'name': 'no results', 'message': '요청한 데이타의 결과가 없습니다.'}, 'results': []}
Address: No results found


In [29]:
df.head(30)

,Unnamed: 0,x,y,address
0,0,127.013725,37.555783,No results found
1,1,127.015149,37.560326,No results found
2,2,127.016254,37.559378,No results found
3,3,127.018157,37.561107,No results found
4,4,127.010355,37.563355,No results found
5,5,127.023495,37.565828,No results found
6,6,127.010717,37.552818,동호로7길 22
7,7,127.017826,37.567531,퇴계로81길 41
8,8,127.012405,37.565122,퇴계로 367
9,9,127.016387,37.564755,다산로 244
